In [1]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel

# Константы
DATA_PATH = "../data/processed/context_answer.csv"
OUTPUT_DIR = "../models"
BATCH_SIZE = 16
MODEL_NAME = 'bert-base-uncased'

In [2]:
# Загрузка предобученной модели и токенизатора
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)

# Переносим модель на GPU, если доступно
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [3]:
def get_bert_embedding(texts, model, tokenizer):
    """Получает эмбеддинги для нескольких текстов с помощью BERT."""
    # Токенизация в батчах
    inputs = tokenizer(
        texts, 
        return_tensors='pt', 
        truncation=True, 
        padding=True, 
        max_length=512
    )
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Переносим данные на GPU

    with torch.no_grad():
        outputs = model(**inputs)

    # Используем среднее значение эмбеддингов токенов как вектор текста
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Переносим обратно на CPU
    return embeddings


def load_data(data_path):
    """Загружает обработанные данные."""
    return pd.read_csv(data_path)


def save_embeddings(embeddings, output_path):
    """Сохраняет эмбеддинги в файл."""
    if embeddings.ndim > 2:
        raise ValueError(f"Ожидался двумерный массив, но получен массив формы {embeddings.shape}")
    np.save(output_path, embeddings)


def load_embeddings(input_path):
    """Загружает эмбеддинги из файла."""
    return np.load(input_path)


def train(data_path, output_dir, batch_size=16):
    """Обучает модель и сохраняет эмбеддинги."""
    # Загружаем данные
    df = load_data(data_path)

    # Получаем эмбеддинги для всех ответов с использованием tqdm и батчей
    response_embeddings = []
    similarities_history = []  # Для хранения истории косинусного сходства

    for i in tqdm(range(0, len(df), batch_size), desc="Создание эмбеддингов"):
        batch_responses = df['response'][i:i + batch_size].tolist()
        embeddings = get_bert_embedding(batch_responses, model, tokenizer)
        response_embeddings.append(embeddings)

        # Примерный контекст для визуализации (можно заменить на реальный)
        example_context = "Why are you late?"
        context_embedding = get_bert_embedding([example_context], model, tokenizer)

        # Вычисляем косинусное сходство для текущего батча
        batch_similarities = cosine_similarity(context_embedding, embeddings)[0]
        similarities_history.extend(batch_similarities)  # Сохраняем историю

    response_embeddings = np.vstack(response_embeddings)  # Объединяем батчи в один массив

    # Проверяем форму массива
    print(f"Форма массива эмбеддингов: {response_embeddings.shape}")

    # Сохраняем эмбеддинги
    os.makedirs(output_dir, exist_ok=True)
    save_embeddings(response_embeddings, os.path.join(output_dir, 'response_embeddings.npy'))
    print(f"Эмбеддинги сохранены в {output_dir}")

    # Визуализация процесса обучения
    plt.figure(figsize=(10, 6))
    plt.plot(similarities_history, label="Косинусное сходство")
    plt.xlabel("Номер батча")
    plt.ylabel("Косинусное сходство")
    plt.title("Изменение косинусного сходства в процессе обучения")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'training_process.png'))  # Сохраняем график
    plt.show()


def infer(context, response_embeddings, df, model, tokenizer):
    """Находит наиболее подходящий ответ для заданного контекста."""
    # Получаем эмбеддинг контекста
    context_embedding = get_bert_embedding([context], model, tokenizer)
    
    # Вычисляем косинусное сходство между контекстом и всеми ответами
    similarities = cosine_similarity(context_embedding, response_embeddings)[0]
    
    # Находим индекс наиболее подходящего ответа
    best_index = np.argmax(similarities)
    return df.iloc[best_index]['response']


In [4]:
# Обучаем модель и сохраняем эмбеддинги
train(DATA_PATH, OUTPUT_DIR, BATCH_SIZE)


Создание эмбеддингов:   0%|          | 3/1305 [00:12<1:33:43,  4.32s/it]


KeyboardInterrupt: 

In [ ]:
# Пример инференса
response_embeddings = load_embeddings(os.path.join(OUTPUT_DIR, 'response_embeddings.npy'))
df = load_data(DATA_PATH)
context = "Why are you late?"
best_response = infer(context, response_embeddings, df, model, tokenizer)

print(f"Контекст: {context}")
print(f"Ответ: {best_response}")


In [ ]:
def validate(response_embeddings, df, model, tokenizer, test_size=0.2, batch_size=16):
    """Оценивает качество модели на тестовой выборке."""
    
    # Определяем устройство (GPU, если доступно, иначе CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Перемещаем модель на устройство

    # Разделяем данные на обучающую и тестовую выборки
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=42)

    # Получаем эмбеддинги для тестовых данных с использованием батчей
    test_embeddings = []
    for i in tqdm(range(0, len(test_df), batch_size), desc="Создание тестовых эмбеддингов"):
        batch_contexts = test_df['context'][i:i + batch_size].tolist()
        
        # Токенизация и перемещение на устройство
        inputs = tokenizer(batch_contexts, return_tensors='pt', padding=True, truncation=True)
        inputs = {key: value.to(device) for key, value in inputs.items()}  # Перемещаем входные данные на устройство
        
        # Получаем эмбеддинги
        with torch.no_grad():
            batch_embeddings = model(**inputs).last_hidden_state.mean(dim=1).cpu().numpy()  # Перемещаем результаты обратно на CPU для дальнейшей обработки
        
        test_embeddings.append(batch_embeddings)
    test_embeddings = np.vstack(test_embeddings)

    # Вычисляем косинусное сходство для тестовых данных
    similarities = cosine_similarity(test_embeddings, response_embeddings)
    best_indices = np.argmax(similarities, axis=1)

    # Оцениваем точность
    correct = 0
    for i, index in enumerate(best_indices):
        if index < len(train_df) and train_df.iloc[index]['response'] == test_df.iloc[i]['response']:
            correct += 1
    accuracy = correct / len(test_df)
    print(f"Точность модели на тестовой выборке: {accuracy:.2f}")

    # Визуализация распределения косинусного сходства
    plt.figure(figsize=(10, 6))
    plt.hist(similarities.flatten(), bins=50, alpha=0.7, label="Косинусное сходство")
    plt.xlabel("Косинусное сходство")
    plt.ylabel("Частота")
    plt.title("Распределение косинусного сходства")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(OUTPUT_DIR, 'validation_histogram.png'))  # Сохраняем график
    plt.show()

    return accuracy

In [ ]:
# Загружаем эмбеддинги и данные
response_embeddings = load_embeddings(os.path.join(OUTPUT_DIR, 'response_embeddings.npy'))
df = load_data(DATA_PATH)

# Пример инференса
context = "Why are you late?"
best_response = infer(context, response_embeddings, df, model, tokenizer)
print(f"Контекст: {context}")
print(f"Ответ: {best_response}")

# Валидация модели
validate(response_embeddings, df, model, tokenizer)